In [1]:
import scipp as sc
from ess import loki, sans
from ess.logging import configure_workflow
import scippneutron as scn
import plopp as pp
from scipy import stats
import numpy as np

In [2]:
logger = configure_workflow('sans2d_I_of_Q', filename='sans2d.log')

In [3]:
# Include effects of gravity?
gravity = True

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')
wavelength_bands = sc.concat(
            [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')

# Q binning
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')

In [4]:
# Using only one-fourth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Sample measurement
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                             spectrum_size=spectrum_size)
# Direct measurement is with the empty sample holder/cuvette
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                             spectrum_size=spectrum_size)

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'
Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised un

In [5]:
data_graph, monitor_graph = sans.i_of_q.make_coordinate_transform_graphs(gravity=gravity)

In [6]:
monitors = {
    'sample': {'incident': sample.attrs["monitor2"].value,
               'transmission': sample.attrs["monitor4"].value},
    'direct': {'incident': direct.attrs["monitor2"].value,
               'transmission': direct.attrs["monitor4"].value}
}

In [7]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation based on the found beam center position 
# TODO: We need to determine this
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
#Testing with not too far off
#x_offset = -0.092 * sc.units.m
#y_offset = 0.083 * sc.units.m

In [8]:
# Add pixel shapes
sample.coords["pixel_width"] = 0.0035 * sc.units.m
sample.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
sample.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
sample.coords["position"].fields.y += bench_pos_y_offset

# Change transmission monitor position
monitors['sample']['transmission'].coords["position"].fields.z += monitor4_pos_z_offset
monitors['direct']['transmission'].coords["position"].fields.z += monitor4_pos_z_offset

In [9]:
# Load direct beam function for main detector
direct_beam = loki.io.load_rkh_wav(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))
direct_beam = sans.i_of_q.resample_direct_beam(
direct_beam=direct_beam,
wavelength_bins=wavelength_bins)

[2022-10-19T15:23:34+0200] WARNING  <scipp> : 
    An interpolation was performed on the direct_beam function. The variances in the direct_beam function have been dropped.


Will have start looping somewhere here

In [10]:
beam_positions = [(-0.094, 0.081), (-0.09288,0.08195), (-0.09,0.08)]

In [11]:
for position in beam_positions:
    print('Starting iteration')
    sample_iter = sample.copy()
    
    x_offset = position[0] * sc.units.m
    y_offset = position[1] * sc.units.m
    
    # Now shift pixels positions to get the correct beam center
    sample_iter.coords['position'].fields.x += x_offset
    sample_iter.coords['position'].fields.y += y_offset
    
    #This part is bit problematic - we can do edges but center will be completely off
    #However the idea is that we are not too far off. 
    mask_edges = (
        (sc.abs(sample_iter.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
        (sc.abs(sample_iter.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

    mask_center = sc.sqrt(
        sample_iter.coords['position'].fields.x**2 +
        sample_iter.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

    sample_iter.masks['edges'] = mask_edges
    sample_iter.masks['center'] = mask_center
    
    mask_tof_min = sc.scalar(13000.0, unit='us')
    mask_tof_max = sc.scalar(15750.0, unit='us')
    tof_masked_region = sc.concat([sample_iter.coords['tof']['tof', 0],
                                   mask_tof_min, mask_tof_max,
                                   sample_iter.coords['tof']['tof', -1]], dim='tof')

    sample_iter = sample_iter.bin(tof=tof_masked_region)
    sample_iter.masks['bragg_peaks'] = sc.array(dims=['tof'], values=[False, True, False])
    
    mask_up = (
       sample_iter.coords['position'].fields.y - y_offset > sc.scalar(-y_offset.value, unit='m')) 
    mask_down = (
       sample_iter.coords['position'].fields.y - y_offset < sc.scalar(-y_offset.value, unit='m')) 
    mask_left = (
       sample_iter.coords['position'].fields.x - x_offset < sc.scalar(-x_offset.value, unit='m')) 
    mask_right = (
       sample_iter.coords['position'].fields.x - x_offset > sc.scalar(-x_offset.value, unit='m')) 
    
    sample_up = sample_iter.copy()
    sample_down = sample_iter.copy()
    sample_left = sample_iter.copy()
    sample_right = sample_iter.copy()
    
    sample_up.masks['mask_down'] = mask_down
    sample_down.masks['mask_up'] = mask_up
    sample_left.masks['mask_right'] = mask_right
    sample_right.masks['mask_left'] = mask_left
    
    #scn.instrument_view(sample_up, pixel_size=0.0075)
    #scn.instrument_view(sample_down, pixel_size=0.0075)
    
    sample_iter, monitors_iter = sans.i_of_q.convert_to_wavelength(
    data=sample_iter,
    monitors=monitors,
    data_graph=data_graph,
    monitor_graph=monitor_graph)
    
    non_background_range = sc.array(dims=['wavelength'], values=[0.7, 17.1], unit='angstrom')
    
    monitors_iter = sans.i_of_q.denoise_and_rebin_monitors(
    monitors=monitors_iter,
    wavelength_bins=wavelength_bins,
    non_background_range=non_background_range)
    
    transmission_fraction = sans.normalization.transmission_fraction(
    data_monitors=monitors_iter['sample'], direct_monitors=monitors_iter['direct'])
    transmission_fraction

    
    solid_angle = sans.normalization.solid_angle_of_rectangular_pixels(
    sample_iter,
    pixel_width=sample_iter.coords['pixel_width'],
    pixel_height=sample_iter.coords['pixel_height'])
    
    quadrants = []
    for quadrant_sample in [sample_up, sample_down, sample_left, sample_right]:

        quadrant_sample, monitors = sans.i_of_q.convert_to_wavelength(
        data=quadrant_sample,
        monitors=monitors,
        data_graph=data_graph,
        monitor_graph=monitor_graph)

        quadrant_sample_q = sans.i_of_q.convert_to_q_and_merge_spectra(
        data=quadrant_sample,
        graph=data_graph,
        wavelength_bands=wavelength_bands,
        q_bins=q_bins,
        gravity=gravity)

        quadrants.append(quadrant_sample_q)
    

    denominator = sans.normalization.compute_denominator(
        direct_beam=direct_beam,
        data_incident_monitor=monitors_iter['sample']['incident'],
        transmission_fraction=transmission_fraction,
        solid_angle=solid_angle)
    
    for coord in ['position', 'sample_position', 'source_position']:
        denominator.coords[coord] = sample_iter.meta[coord]
     
    #Denominator seems to change slightly between different positions, however we should test how much does it matter
    denominator_q = sans.i_of_q.convert_to_q_and_merge_spectra(
        data=denominator,
        graph=data_graph,
        wavelength_bands=wavelength_bands,
        q_bins=q_bins,
        gravity=True)
    
    
    sample_normalized_up = sans.normalization.normalize(
            numerator=quadrants[0],
            denominator=denominator_q)

    sample_normalized_down = sans.normalization.normalize(
            numerator=quadrants[1],
            denominator=denominator_q)

    sample_normalized_left = sans.normalization.normalize(
            numerator=quadrants[2],
            denominator=denominator_q)

    sample_normalized_right = sans.normalization.normalize(
            numerator=quadrants[3],
            denominator=denominator_q)
    
    #qvalues=sc.midpoints(result.coords['Q']).values 
    Iup=sample_normalized_up.hist().values
    Idown=sample_normalized_down.hist().values
    Ileft=sample_normalized_left.hist().values
    Iright=sample_normalized_right.hist().values
    res_up_down = stats.linregress(Iup, Idown)
    res_up_down_2 = res_up_down.rvalue**2
    res_left_right = stats.linregress(Ileft, Iright)
    res_left_right_2 = res_left_right.rvalue**2
    final_res = np.sqrt(res_up_down_2**2 + res_left_right_2**2)
    print(f"R2 score (up_down){res_up_down_2}")
    print(f"R2 score (left_right){res_left_right_2}")
    print(f"R2 score (summed){final_res}")
    #sc.plot({'up':sample_normalized_up,'down':sample_normalized_down,
    #         'left':sample_normalized_left , 'right':sample_normalized_right})

Starting iteration


'histogram' is deprecated. Use 'hist' instead.


R2 score (up_down)0.9969345384794037
R2 score (left_right)0.9975200542616156
R2 score (summed)1.4102924280684621
Starting iteration


'histogram' is deprecated. Use 'hist' instead.


R2 score (up_down)0.995518183294234
R2 score (left_right)0.9985006869352496
R2 score (summed)1.4099858421557352
Starting iteration


'histogram' is deprecated. Use 'hist' instead.


R2 score (up_down)0.9980387465551326
R2 score (left_right)0.9989223002295167
R2 score (summed)1.412064836160567


Here we start sample dependent part

In [12]:
sample_normalized_right.hist()

<scipp.DataArray>
Dimensions: Sizes[Q:140, ]
Coordinates:
  Q                         float64           [1/Å]  (Q [bin-edge])  [0.01, 0.0142143, ..., 0.595786, 0.6]
  pixel_height              float64              [m]  ()  [0.00203398]
  pixel_width               float64              [m]  ()  [0.0035]
Data:
                            float32  [dimensionless]  (Q)  [43.124, 61.3444, ..., 0, 0]  [0.235134, 0.247705, ..., 0, 0]
Attributes:
  Changer                 DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:2, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000]
Data:
                             string  [dimensionless]  (time)  ["IT", "JT"]

]
  Det_Temp_FLB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [16.8, 16.8, 16.8]

]
  Det_Temp_FLT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_FRB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_FRT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_RLB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.8, 17.8, 17.8]

]
  Det_Temp_RLT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Det_Temp_RRB            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.3, 17.3, 17.3]

]
  Det_Temp_RRT            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:3, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T21:08:13.000000000]
Data:
                            float64  [dimensionless]  (time)  [17.4, 17.4, 17.4]

]
  Fast_Shutter            DataArray        <no unit>  ()  [<scipp.DataArray>
Dimensions: Sizes[time:4, ]
Coordinates:
  time                    datetime64             [ns]  (time)  [2020-02-12T20:37:41.000000000, 2020-02-12T20:37:42.000000000, 2020-02-12T20:38:00.000000000, 2020-02-12T20:38:13.000000000]
Data